# DEUXIEME PARTIE DU TP

## THEME : Etude du jeu de données Parkinson_Speech

## Importation des librairies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Chargement des données

In [ ]:
Parkinson_DF = pd.read_csv('train_data.txt', sep = ',', 
names=['Subject id','Jitter(local)','Jitter(local, absolute)','Jitter(rap)','Jitter(ppq5)','Jitter(ddp)',
       'Shimmer(local)','Shimmer(local, dB)','Shimmer(apq3)','Shimmer(apq5)','Shimmer(apq11)','Shimmer(dda)',
       'AC','NTH','HTN','Median pitch','Mean pitch','Standard deviation','Minimum pitch','Maximum pitch',
       'Number of pulses','Number of periods','Mean period','Standard deviation of period',
       'Fraction of locally unvoiced frames','Number of voice breaks','Degree of voice breaks',
       'UPDRS','class information'])


In [ ]:
Parkinson_DF.head()

## Analyse statistique descriptive

Examinons les données brutes pour comprendre le nombre d'enregistrements, la structure du fichier, le nombre d'attributs, les types d'attributs et les défis probables dans l'ensemble de données. Nous ferons également quelques observations à ce sujet.

### Profilage des donnees

In [ ]:
Parkinson_DF.info()

- On peut s'apercevoir que le jeu de donnees ne presente pas de valeurs nulles.
- on peut noter aussi que nous avons 1040 entrées dans notre DataSet
- Nous avons 29 colonnes dont 23 de types float et 6 integer (valeur entière)
- Nous n'avons pas de variables categoricielle

L'ensemble de données contient les variables suivantes :

Les variables numériques sont les suivantes :

Subject id: Identification du sujet, Jitter(local),Jitter(local absolue),jitter(RAP),jitter(PPQ5),Jitter(DDP): se sont plusieurs mesures de la variation de la fréquence fondamentale.
Shimmer(local),Shimmer(local,dB),Shimmer(APQ3),Shimmer(APQ5),Shimmer(APQ11),Shimmer(DDA) : Plusieurs mesures de la variation de l'amplitude.
NHT, HNT :  Deux mesures du rapport entre le bruit et les composantes tonales de la voix.
AC : Anticorps.
Median pitch,Maximum pitch,Mean pitch,Minimum pitch,Standard deviation : Plusieurs mesure relatives à la pente 
Number of pulses,Number of periods,Mean period,Standard deviation of period: Plusieurs mesures de la periode 
Fraction of locally unvoiced frames: Mesure de la fraction des trames localement non voisées.
Number of voice breaks, Degree of voice breaks: Deux mesures liées à la variation de la voix.
class information : information relative à la classe du patient
UPDRS :Unified Parkinson's Disease Rating Scale est notre variable cible

In [ ]:
dupes=Parkinson_DF.duplicated()

In [ ]:
print('Le nombre de doublons dans lensemble de données est le suivant:',sum(dupes),'\n','Il est donc évident quil n y a pas de doublons dans lensemble de données.')

In [ ]:
Parkinson_DF.describe()

Commentaire:

- Pour les colonnes Maximum pitch, Number of pulses, Number of periods, on note une dispersion assez elevee des donnees par rapport à la moyenne.
- Par rapport aux quartiles, la repartition des donnees est assez differentes selon les colonnes. En effet, pour les colonnes , Maximum pitch,Number_of_pulses, Number of pulses et Number of periods on note plus une repartition assez homogene des donnees sur q1, q2 et q3 avec une des proportions croissantes passant de q1 , q2 et q3. Pour d'autres colonnes comme Mean_period, Standard_deviation of period, Jitter(local, absolute),Jitter(rap),Jitter(ppq5) la tendance de la repartition est differentes des colonnes citées ci-dessus. 
- S'agissant de la moyenne, Il est notable de remarquer l'heterogeneité des échelles. En effet, les moyennes des colonnes du Dataset sont tres differentes : Ceci peut etre problematique durant la phase d'apprentissage ; ainsi, un travail de normalisation sera fait pour contourner ce probleme.



In [ ]:
cname=Parkinson_DF.columns
data_desc=Parkinson_DF.describe().T
data_desc['Skewness']=round(Parkinson_DF[cname].skew(),4)
pd.DataFrame(data_desc)

Commentaire: 
Nous avons également ajouté une colonne d'asymétrie pour comprendre l'asymétrie de chaque attribut qui peut aussi être facilement comparée à la moyenne et à la médiane.
Les données semblent être asymétriques pour Jitter(local), Jitter(local,absolu), jitter(RAP), jitter(PPQ5), Jitter(DDP), Shimmer(APQ3), NHR, Shimmer(dB); ce qui est également évident du fait que la moyenne et la médiane pour ces attributs sont assez éloignées. Il pourrait également y avoir des écarts potentiels que nous évaluerons plutard
Pour les attributs : Shimmer (local), Shimmer(APQ3), Shimmer(APQ5), Shimmer(DDA) nous constatons qu'il y a une légère asymétrie.

## Correlation des donnees

In [ ]:
# verification de la correlation entres les variables independantes
usecols =[i for i in Parkinson_DF.columns if i != ['Subject id ','UPDRS']]
sns.pairplot(Parkinson_DF[usecols]);

- Commentaire : 
Compte tenu du nombre d'attributs tres important, le Pairplot est peu visible.Apparement, Il semble y avoir une corrélation entre les attributs mais il est difficile de l'identifier visuellement. Nous allons donc calculer la corrélation entre les variables.

In [ ]:
correlation=Parkinson_DF[usecols].corr()
fig, ax = plt.subplots(figsize=(20,20))
sns.heatmap(correlation,annot=True,linewidth=0.05,ax=ax, fmt= '.2f');

Commentaire : 
- Il y a __29 attributs__ caracterisant les echantillons de nos sujet. Analysons l'ensemble des données. </br>Nous allons adopter l'approche permettant à analyser les attributs.</br>Nous allons effectuer les analyses suivantes sur les attributs qui nous semble pertinent (sauf l'id du sujet et le UPDRS - variable cible).


- Vérification de l'asymétrie
- Distributipn des donnés
- Co-relation avec d'autres attributs

Pour la variable cible :

Comparaison entre l'UPDRS du sujet et d'autres attributs en utilisant différents tracés
SUJET ID et class Information sont des attributs redondants, et puisqu'aucune analyse ne peut être effectuée sur ces variables et qu'il ne sont pas non plus pertinent pour le modèle. Nous comptons les supprimer de notre jeu de données.

##### Epuration du jeu de données

In [ ]:
Parkinson_DF=Parkinson_DF.drop('Subject id', axis=1)

In [ ]:
Parkinson_DF.drop('class information', axis=1)

In [ ]:
#Verification de l'asymetrie
Jitter(local),Jitter(local absolue),jitter(RAP),jitter(PPQ5),Jitter(DDP)

In [ ]:
Atr1='Shimmer(local)'
Atr2='Shimmer(apq11)'
Atr3='Shimmer(dda)'

In [ ]:
Atr1_pt=Parkinson_DF.describe().loc[['min','25%','50%','75%','max'],[Atr1]]
Atr2_pt=Parkinson_DF.describe().loc[['min','25%','50%','75%','max'],[Atr2]]
Atr3_pt=Parkinson_DF.describe().loc[['min','25%','50%','75%','max'],[Atr3]]

summ_ = pd.concat([Atr1_pt,Atr2_pt,Atr3_pt],axis=1,sort=False)

print('resultat:','\n','\n',summ_)

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(16,8)) 
sns.distplot(Parkinson_DF[Atr1],ax=ax[0]) 
sns.distplot(Parkinson_DF[Atr2],ax=ax[1])
sns.distplot(Parkinson_DF[Atr3],ax=ax[2])

Commentaire :
    Nous constatons une distribution assez homogenes des variables qui presentaient une asymetriques assez importante

## Exploration des données

#### Exploration uni-dimensionelle

# Data visualisation

##### Detection des outliers

In [ ]:
plt.figure(figsize=(25,5))
plt.subplot(1,4,1)
sns.boxplot(x='UPDRS',y='Jitter(local)', data=Parkinson_DF)
plt.subplot(1,4,2)
sns.boxplot(x='UPDRS',y='Jitter(local, absolute)', data=Parkinson_DF)
plt.subplot(1,4,3)
sns.boxplot(x='UPDRS',y='Jitter(rap)', data=Parkinson_DF)
plt.subplot(1,4,4)
sns.boxplot(x='UPDRS',y='Jitter(ddp)', data=Parkinson_DF);

commentaire :
On voit que la plus part des variables (Jitter(local),Jitter(local absolue),jitter(RAP),jitter(PPQ5),Jitter(DDP) ) qui mesurent la variation de la fréquence fondamentale presentent des valeurs aberrentes.

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows = 4, ncols = 4)

i_name = 0
            
for i in range(4):
    for j in range(4):
            i_name = i_name+1
            name_col = Parkinson_DF.columns[i_name]
            axs[i, j].hist(Parkinson_DF[name_col])
            axs[i, j].set_title(name_col)

fig.tight_layout()
plt.show()

commentaire:
on constate une distribution heterogene des données

In [ ]:
AtrNTH='NTH'
Atr2HTN='HTN'

fig, ax = plt.subplots(1,2,figsize=(16,8))
sns.boxplot(x='UPDRS',y=AtrNTH,data=Parkinson_DF,ax=ax[0])
sns.boxplot(x='UPDRS',y=Atr2HTN,data=Parkinson_DF,ax=ax[1])

commentaire:
on constate une distribution des données abberentes plus importante sur le HTN par rapport au score UPDRS

### Transformation des données

In [ ]:
from math import sqrt, log

Parkinson_DF["Jitter(local, absolute)"] = Parkinson_DF["Jitter(local, absolute)"].map(lambda x: sqrt(x))
Parkinson_DF["Jitter(ppq5)"] = Parkinson_DF["Jitter(ppq5)"].map(lambda x: log(x))
Parkinson_DF["Shimmer(apq5)"] = Parkinson_DF["Shimmer(apq5)"].map(lambda x: log(x))

In [ ]:
Parkinson_DF.head()

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(Parkinson_DF[['Jitter(local)','Jitter(local, absolute)','Jitter(rap)','Jitter(ppq5)','Jitter(ddp)',
       'Shimmer(local)','Shimmer(local, dB)','Shimmer(apq3)','Shimmer(apq5)','Shimmer(apq11)','Shimmer(dda)',
       'AC','NTH','HTN','Median pitch','Mean pitch','Standard deviation','Minimum pitch','Maximum pitch',
       'Number of pulses','Number of periods','Mean period','Standard deviation of period',
       'Fraction of locally unvoiced frames','Number of voice breaks','Degree of voice breaks',
       'UPDRS']], alpha=0.2, figsize=(15, 15), diagonal='kde')
plt.show()

# Apprentissage du modele 

### Création de l'ensemble d'apprentissage / validation et de l'ensemble de test

In [ ]:
Parkinson_DF=Parkinson_DF.drop('class information', axis=1)

In [ ]:
X = Parkinson_DF.drop('UPDRS', axis=1)
Y = Parkinson_DF['UPDRS']

In [ ]:
X.head()

In [ ]:
Y.head()

In [ ]:
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler  

X_1, X_2, Y_1, Y_2 = train_test_split(X, Y, train_size = 0.8, test_size = 0.2, random_state = 1)

X_av = X_1.to_numpy()
X_t = X_2.to_numpy()
Y_av = np.transpose([Y_1.to_numpy()])
Y_t = np.transpose([Y_2.to_numpy()])

## 1. Régression linéaire ordinaire

In [ ]:
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
# instanciation de mon regression lineaire
reg_lin = linear_model.LinearRegression()
reg_app = reg_lin.fit(X_av, Y_av)
Y_reg_pred_test = reg_lin.predict(X_t)


print("Coefficient de régression Lineaire : \n", reg_lin.coef_)
print()

print("Erreur quadratique (TestingSet) : %.2f" % mean_squared_error(Y_t, Y_reg_pred_test))
print("Coefficient de détermination (TestingSet) : %.2f" % r2_score(Y_t, Y_reg_pred_test))

In [ ]:
coef = pd.Series(reg_app.coef_[0], index =  X.columns)
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("regression lineaire")
plt.show()

##  Régression linéaire pénalisée : Ridge

### validation-croisée pour la sélection de alpha

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [ ]:
#Instanciation du model
ridge_reg = linear_model.Ridge()

#definition des valeur de alpha
params_Ridge = {'alpha': [10, 1, 0.1, 0.01, 0.001, 0.0001] , "fit_intercept": [True, False]}


#Recherche de meilleur parametre
Ridge_GS = GridSearchCV(ridge_reg,
                        param_grid=params_Ridge,
                       )
#Apprentissage du model
ridge = Ridge_GS.fit(X_av,Y_av)

scores = Ridge_GS.cv_results_["mean_test_score"]
scores_std = Ridge_GS.cv_results_["std_test_score"]

print("Résultats de la validation Croisé sur l'ensemble des paramètres:")   
print("Meilleurs paramètres alpha:")
print(Ridge_GS.best_params_)

In [ ]:
scores_std

In [ ]:
scores

In [ ]:
new_best_model = Ridge_GS.best_estimator_

In [ ]:
New_Y_t_pred = new_best_model.predict(X_t)

In [ ]:
print("Erreur quadratique (TestingSet) : %.2f" % mean_squared_error(Y_t, New_Y_t_pred))
print("Coefficient de détermination (TestingSet) : %.2f" % r2_score(Y_t, New_Y_t_pred))

## Régression linéaire pénalisée : Lasso

In [ ]:
from sklearn import datasets
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [ ]:
lasso = linear_model.Lasso()

#definition des valeur de alpha
params_Lasso = {'alpha': [10, 1, 0.1, 0.01, 0.001, 0.0001] , "fit_intercept": [True, False]}


#Recherche de meilleur parametre
Lasso_GS = GridSearchCV(lasso,
                        param_grid=params_Lasso,
                       )
#Apprentissage du model
lasso = Lasso_GS.fit(X_av, Y_av)

scores = Lasso_GS.cv_results_["mean_test_score"]
scores_std = Lasso_GS.cv_results_["std_test_score"]

print("Résultats de la validation Croisé (Lasso) sur l'ensemble des paramètres:")   
print("Meilleurs paramètres alpha du Lasso :")
print(Lasso_GS.best_params_)

In [ ]:
best_model = Lasso_GS.best_estimator_

In [ ]:
Y_t_pred = best_model.predict(X_t)

In [ ]:
print("Erreur quadratique (TestingSet) : %.2f" % mean_squared_error(Y_t, Y_t_pred))
print("Coefficient de détermination (TestingSet) : %.2f" % r2_score(Y_t, Y_t_pred))